In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import os
import csv 
import math
from PIL import Image
from scipy import ndimage

def load_dataset_2():
    with h5py.File('heart.h5', "r") as train_dataset:
        train_set_x_orig = np.array(train_dataset["train_set_x"][:])
        train_set_y_orig = np.array(train_dataset["train_set_y"][:])

    with h5py.File('heart.h5', "r") as test_dataset:
        test_set_x_orig = np.array(test_dataset["test_set_x"][:])
        test_set_y_orig = np.array(test_dataset["test_set_y"][:])
        classes = np.array(test_dataset["list_classes"][:])

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

def load_set():
    file = open("heart2.csv")
    numpy_array = np.loadtxt(file, delimiter=",")
    #print (numpy_array.T)
    numpy_array_transpose = numpy_array.T 
    m_train = math.ceil (0.8 * numpy_array_transpose.shape[1])
    #m_set = len(numpy_array_transpose) - m_train
    #print ("mtrain is",m_train)
    print(numpy_array_transpose.shape)
    train_set_x_orig = numpy_array_transpose[0:numpy_array_transpose.shape[0]-1,0:m_train]
    train_set_y_orig = numpy_array_transpose[numpy_array_transpose.shape[0]-1:numpy_array_transpose.shape[0],0:m_train]
    #train_set_y_orig = numpy_array_transpose[13:14,0:820]
    test_set_x_orig = numpy_array_transpose [0:numpy_array_transpose.shape[0]-1, m_train:numpy_array_transpose.shape[1]]
    test_set_y_orig = numpy_array_transpose[numpy_array_transpose.shape[0]-1 : numpy_array_transpose.shape[0],m_train:numpy_array_transpose.shape[1]]
    #print (train_set_x_orig)
    #print ("here is y" , len(train_set_y_orig))
    #print (train_set_y_orig)
    #print ("here is test set")
    #print (test_set_x_orig)
    #print ("here is test y", len(test_set_y_orig))
    #print (test_set_y_orig)
    #print(train_set_x_orig.shape)
    #print(train_set_y_orig.shape)
    #print(train_set_y_orig)
   #print(test_set_x_orig.shape)
    #print(test_set_y_orig.shape)
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig


from scipy.special import expit

def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    ### START CODE HERE ### (≈ 1 line of code)
   # s = 1 / (1 + np.exp(-z))
    s = expit(z)
    ### END CODE HERE ###
    
    return s   

In [3]:
os.getcwd()
load_set()

(14, 1025)


(array([[52., 53., 70., ..., 45., 42., 58.],
        [ 1.,  1.,  1., ...,  1.,  1.,  0.],
        [ 0.,  0.,  0., ...,  0.,  3.,  0.],
        ...,
        [ 2.,  0.,  0., ...,  2.,  2.,  1.],
        [ 2.,  0.,  0., ...,  0.,  2.,  2.],
        [ 3.,  3.,  3., ...,  2.,  2.,  1.]]),
 array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
         1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
         0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
         1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
         1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1

In [4]:
def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    w = np.zeros((dim, 1))
    b = 0
    ### END CODE HERE ###

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

dim = 2
w, b = initialize_with_zeros(dim)
print ("w = " + str(w))
print ("b = " + str(b))

w = [[0.]
 [0.]]
b = 0


In [5]:
def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = X.shape[1]
    
    # FORWARD PROPAGATION (FROM X TO COST)
    ### START CODE HERE ### (≈ 2 lines of code)

    # compute activation 
    A = sigmoid(np.dot(X.T,w) + b)          
    #print(A)
    cost = -(1/m) * np.sum(Y.T * np.log(A) + (1 - Y.T) * (np.log(1-A)) )                                 # compute cost
    ### END CODE HERE ###
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    ### START CODE HERE ### (≈ 2 lines of code)
    
    dw = (1/m) * np.dot(X,(A-Y.T))

    db = (1/m) * np.sum(A-Y.T)
    
    ### END CODE HERE ###
   # print (dw.shape)
    #print (w.shape)
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

w, b, X, Y = np.array([[1],[2]]), 2, np.array([[1,2],[3,4]]), np.array([[1,0]])
#grads, cost = propagate(w, b, X, Y)
#print ("dw = " + str(grads["dw"]))
#print ("db = " + str(grads["db"]))
#print ("cost = " + str(cost))   

train_set_x, train_set_y, test_set_x, test_set_y= load_set()
w, b = initialize_with_zeros(train_set_x.shape[0])
#rint ("w is ", w )
#print (train_set_x.shape)
#print(np.dot(train_set_x.T,w))
grads, cost = propagate(w, b, train_set_x, train_set_y)
print(cost)

(14, 1025)
0.6931471805599453


In [6]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propagate(w, b, X, Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w - learning_rate * dw
        b = b - learning_rate * db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training examples
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

train_set_x, train_set_y, test_set_x, test_set_y= load_set()
w, b = initialize_with_zeros(train_set_x.shape[0])
params, grads, costs = optimize(w, b, train_set_x, train_set_y, num_iterations= 200, learning_rate = 0.009, print_cost = 1)

print ("w = " + str(params["w"]))
print ("b = " + str(params["b"]))
print ("dw = " + str(grads["dw"]))
print ("db = " + str(grads["db"]))


(14, 1025)
Cost after iteration 0: 0.693147
Cost after iteration 100: nan
w = [[-1.29295534]
 [-0.12274005]
 [ 0.35982242]
 [-2.46184091]
 [-1.72467974]
 [-0.01389658]
 [ 0.04531177]
 [ 4.44546661]
 [-0.16413032]
 [-0.41018506]
 [ 0.13857896]
 [-0.33941075]
 [-0.1906794 ]]
b = -0.0074245814519839
dw = [[2.15335096e+01]
 [3.22999531e-01]
 [1.95619352e-01]
 [5.08426895e+01]
 [9.62393588e+01]
 [6.58536584e-02]
 [1.58764746e-01]
 [5.64655158e+01]
 [2.13243434e-01]
 [6.17784289e-01]
 [4.77020504e-01]
 [4.31611976e-01]
 [9.82676600e-01]]
db = 0.38576016799938095


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


In [7]:

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    ### START CODE HERE ### (≈ 1 line of code)
    A = sigmoid(np.dot(X.T,w) + b)
    ### END CODE HERE ###
    print(range(A.shape[1]))
    for i in range(A.shape[0]):        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        ### START CODE HERE ### (≈ 4 lines of code)
        #pass
        if(A[i,0] > 0.5):
            Y_prediction[0,i] = 1
        else:
            Y_prediction[0,i] = 0
        ### END CODE HERE ###
        
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

print ("predictions = " + str(predict(w, b, X)))
print("X is " + str(X.shape))

ValueError: ignored

In [8]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """
    
    ### START CODE HERE ###
    
    # initialize parameters with zeros (≈ 1 line of code)

    w, b = initialize_with_zeros(X_train.shape[0])
    print(w.shape)
    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w,b, X_test)
    Y_prediction_train = predict(w,b, X_train)

    ### END CODE HERE ###

    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

train_set_x, train_set_y, test_set_x, test_set_y= load_set()
d = model(train_set_x, train_set_y, test_set_x, test_set_y, num_iterations = 5000000, learning_rate = 0.00001, print_cost = False)

(14, 1025)
(13, 1)
range(0, 1)
range(0, 1)
train accuracy: 86.46341463414635 %
test accuracy: 79.51219512195122 %
